In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas.tseries import offsets
from skforecast import __version__
print(__version__)


0.10.1


In [3]:
# simulate a time series
n = 365
y = pd.Series(
    data=np.arange(n),
    index = pd.date_range(start='2000-01-01', periods=n, freq='d')
)
y

2000-01-01      0
2000-01-02      1
2000-01-03      2
2000-01-04      3
2000-01-05      4
             ... 
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Length: 365, dtype: int32

In [4]:
forecaster = ForecasterEquivalentDate(offset=10, n_offsets=2)
forecaster.fit(y=y)
forecaster

ForecasterEquivalentDate 
Offset: 10 
Number of offsets: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-11-12 00:18:43 
Last fit date: 2023-11-12 00:18:43 
Skforecast version: 0.10.1 
Python version: 3.10.11 
Forecaster id: None 

In [5]:
forecaster.predict(steps=15)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
Freq: D, Name: pred, dtype: float64

In [6]:
metric, predictions_1 = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=20,
    steps=15,
    metric='mean_absolute_error',
    verbose=False
)

  0%|          | 0/23 [00:00<?, ?it/s]

In [7]:
forecaster = ForecasterEquivalentDate(
    offset=offsets.Day(10),
    n_offsets=2
)
forecaster.fit(y=y)
forecaster

ForecasterEquivalentDate 
Offset: <10 * Days> 
Number of offsets: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-11-12 00:18:46 
Last fit date: 2023-11-12 00:18:46 
Skforecast version: 0.10.1 
Python version: 3.10.11 
Forecaster id: None 

In [8]:
forecaster.predict(steps=15)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
Freq: D, Name: pred, dtype: float64

In [9]:
forecaster = ForecasterEquivalentDate(
    offset = offsets.Day(10),
    n_offsets=2
)

print(forecaster.regressor)

metric, predictions_2 = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=20,
    steps=15,
    metric='mean_absolute_error',
    refit=True,
    verbose=False,
    n_jobs='auto'
)
predictions_2

None


  0%|          | 0/23 [00:00<?, ?it/s]

,pred
2000-01-21,5.0
2000-01-22,6.0
2000-01-23,7.0
2000-01-24,8.0
2000-01-25,9.0
...,...
2000-12-26,335.0
2000-12-27,336.0
2000-12-28,337.0
2000-12-29,338.0


In [10]:
pd.testing.assert_frame_equal(predictions_1, predictions_2)

In [38]:
last_window_values = y.iloc[-20:].to_numpy()
last_window_values

array([345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357,
       358, 359, 360, 361, 362, 363, 364])

In [45]:
offset = 7
n_offsets = 1
steps = 5

equivalent_indexes = np.tile(
                        np.arange(-offset, 0),
                        int(np.ceil(steps/offset))
                    )

equivalent_indexes

array([-7, -6, -5, -4, -3, -2, -1])

In [46]:
# When n_offsets == 1
equivalent_indexes = equivalent_indexes[:steps]
print(equivalent_indexes)
equivalent_values = last_window_values[equivalent_indexes]
print(equivalent_values)
equivalent_values.ravel()

[-7 -6 -5 -4 -3]
[358 359 360 361 362]


array([358, 359, 360, 361, 362])

In [47]:
offset = 7
n_offsets = 2
steps = 5

equivalent_indexes = np.tile(
                        np.arange(-offset, 0),
                        int(np.ceil(steps/offset))
                    )

equivalent_indexes

array([-7, -6, -5, -4, -3, -2, -1])

In [48]:
# When n_offsets > 1
equivalent_indexes = [equivalent_indexes - n * offset 
                      for n in np.arange(n_offsets)]
print(equivalent_indexes)
equivalent_indexes = np.vstack(equivalent_indexes)
print(equivalent_indexes)
equivalent_values = last_window_values[equivalent_indexes]
print(equivalent_values)
predictions = np.apply_along_axis(
                np.mean,
                axis = 0,
                arr  = equivalent_values
            )
print(predictions)

[array([-7, -6, -5, -4, -3, -2, -1]), array([-14, -13, -12, -11, -10,  -9,  -8])]
[[ -7  -6  -5  -4  -3  -2  -1]
 [-14 -13 -12 -11 -10  -9  -8]]
[[358 359 360 361 362 363 364]
 [351 352 353 354 355 356 357]]
[354.5 355.5 356.5 357.5 358.5 359.5 360.5]


In [50]:
last_window_values

array([345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357,
       358, 359, 360, 361, 362, 363, 364])

In [52]:
y.index[-20:]

DatetimeIndex(['2000-12-11', '2000-12-12', '2000-12-13', '2000-12-14',
               '2000-12-15', '2000-12-16', '2000-12-17', '2000-12-18',
               '2000-12-19', '2000-12-20', '2000-12-21', '2000-12-22',
               '2000-12-23', '2000-12-24', '2000-12-25', '2000-12-26',
               '2000-12-27', '2000-12-28', '2000-12-29', '2000-12-30'],
              dtype='datetime64[ns]', freq='D')

In [89]:
offset = pd.offsets.Day(6)
n_offsets = 2
steps = 5

In [88]:
last_window_values = y.iloc[-11:].to_numpy()
last_window_values

array([354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364])

In [90]:
from skforecast.utils import expand_index

predictions_index = expand_index(index=y.index[-11:], steps=steps)
print(predictions_index)
max_allowed_date = y.index[-11:][-1]
print(max_allowed_date)

DatetimeIndex(['2000-12-31', '2001-01-01', '2001-01-02', '2001-01-03',
               '2001-01-04'],
              dtype='datetime64[ns]', freq='D')
2000-12-30 00:00:00


In [91]:
offset_dates = []
for date in predictions_index:
    selected_offsets = []
    while len(selected_offsets) < n_offsets:
        offset_date = date - offset
        if offset_date <= max_allowed_date:
            selected_offsets.append(offset_date)
        date = offset_date
    offset_dates.append(selected_offsets)

offset_dates = np.array(offset_dates)
print(offset_dates)

[[Timestamp('2000-12-25 00:00:00') Timestamp('2000-12-19 00:00:00')]
 [Timestamp('2000-12-26 00:00:00') Timestamp('2000-12-20 00:00:00')]
 [Timestamp('2000-12-27 00:00:00') Timestamp('2000-12-21 00:00:00')]
 [Timestamp('2000-12-28 00:00:00') Timestamp('2000-12-22 00:00:00')]
 [Timestamp('2000-12-29 00:00:00') Timestamp('2000-12-23 00:00:00')]]


In [92]:
offset_dates.ravel()

array([Timestamp('2000-12-25 00:00:00'), Timestamp('2000-12-19 00:00:00'),
       Timestamp('2000-12-26 00:00:00'), Timestamp('2000-12-20 00:00:00'),
       Timestamp('2000-12-27 00:00:00'), Timestamp('2000-12-21 00:00:00'),
       Timestamp('2000-12-28 00:00:00'), Timestamp('2000-12-22 00:00:00'),
       Timestamp('2000-12-29 00:00:00'), Timestamp('2000-12-23 00:00:00')],
      dtype=object)

In [93]:
equivalent_values = (
    y.iloc[-11:,].
    reindex(offset_dates.ravel())
    .to_numpy()
    .reshape(-1, n_offsets)
)
equivalent_values

array([[359.,  nan],
       [360., 354.],
       [361., 355.],
       [362., 356.],
       [363., 357.]])

In [94]:
equivalent_values = pd.DataFrame(
                                    data    = equivalent_values,
                                    index   = predictions_index,
                                    columns = [f'offset_{i}' 
                                               for i in range(n_offsets)]
                                )
equivalent_values

,offset_0,offset_1
2000-12-31,359.0,NaN
2001-01-01,360.0,354.0
2001-01-02,361.0,355.0
2001-01-03,362.0,356.0
2001-01-04,363.0,357.0


In [97]:
incomplete_offsets = equivalent_values.isnull().any(axis=1)
incomplete_offsets = incomplete_offsets[incomplete_offsets].index
incomplete_offsets

DatetimeIndex(['2000-12-31'], dtype='datetime64[ns]', freq='D')

In [98]:
(f"Steps: {incomplete_offsets.strftime('%Y-%m-%d').to_list()} "
f"are calculated with less than {n_offsets} offsets. To avoid "
f"this increase `last_window` size or decrease the number "
f"of offsets. The current configuration needs a total offset "
f"of {offset * n_offsets}.")

"Steps: ['2000-12-31'] are calculated with less than 2 offsets. To avoid this increase `last_window` size or decrease the number of offsets. The current configuration needs a total offset of <12 * Days>."

In [95]:
aggregate_values = equivalent_values.apply(np.mean, axis=1)
aggregate_values

2000-12-31    359.0
2001-01-01    357.0
2001-01-02    358.0
2001-01-03    359.0
2001-01-04    360.0
Freq: D, dtype: float64